In [1]:
import datetime #date time
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import sys
customized_path = "/content/drive/Shareddrives/Urban Computing Project/"
sys.path.append(customized_path)

In [4]:
%cd /content/drive/Shareddrives/Urban\ Computing\ Project/datasets

/content/drive/Shareddrives/Urban Computing Project/datasets


In [5]:
!ls

archive.zip		       mta_1712.csv
atstop_zone.csv		       next_stop_locations.csv
AugustCleaned.csv	       next_stop_possibility.csv
AugustCleanedV2.csv	       next_stop_possibility_with_google.csv
AugustCleanedV3.csv	       next_stop_with_zone.csv
computed_geocodes.csv	       OctoberCleaned.csv
corrected_google_lat_long.csv  OctoberCleanedV2.csv
DecemberCleaned.csv	       OctoberCleanedV3.csv
DecemberCleanedV2.csv	       taxi_agg_bq.csv
juneCleaned.csv		       taxi_zones.dbf
JuneCleaned.csv		       taxi_zones.prj
juneCleanedV2.csv	       taxi_zones.sbn
JuneCleanedV2.csv	       taxi_zones.sbx
juneCleanedV3.csv	       taxi_zones.shp
JuneCleanedV3.csv	       taxi_zones.shp.xml
missing_after_google.csv       taxi_zones.shx
mta_1706.csv		       taxi_zones.zip
mta_1708.csv		       yellow_tripdata_2017-06.csv
mta_1710.csv


In [6]:
names = range(0, 19)
transit = pd.read_csv("mta_1712.csv",header=None,names=names, low_memory=False)
transit.shape

(6461754, 19)

## Cleaning mta before loading to bigquery


In [7]:
transit.describe(include='all')

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
count,6461754,6160516,6461754,6069060,6069060,6069060,6461754,6145591,6145591,6461754,6461754,6461754,6153673,6454661,6454661,5398612,5965347,362,0.0
unique,245730,3,329,619,680,680,809,543,543,5710,361326,425761,11021,213,16877,726850,93087,153,NaN
top,2017-12-29 13:31:47,1,B6,MERRICK BL/ARCHER AV,40.704907,-73.793306,WASHINGTON HTS GW BRIDGE,40.704933,-73.793318,NYCT_4606,40.807583,-73.831286,MERRICK BL/ARCHER AV,approaching,0,2017-12-29 10:31:52,18:00:00,16:57:08,NaN
freq,210,3093867,122391,87060,87060,87060,84246,88379,88379,2794,3612,3605,12102,2422385,386319,1227,2329,8,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
transit.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,RecordedAtTime,DirectionRef,PublishedLineName,OriginName,OriginLat,OriginLong,DestinationName,DestinationLat,DestinationLong,VehicleRef,VehicleLocation.Latitude,VehicleLocation.Longitude,NextStopPointName,ArrivalProximityText,DistanceFromStop,ExpectedArrivalTime,ScheduledArrivalTime,NaN,NaN
1,2017-12-01 00:05:49,0,B67,MC DONALD AV/CORTELYOU RD,40.63816,-73.978939,DNTWN BKLYN SANDS ST via FLATBUSH AV,40.700253,-73.98703,NYCT_406,40.671578,-73.977672,7 AV/GARFIELD PL,approaching,130,2017-12-01 00:06:21,24:01:30,NaN,NaN
2,2017-12-01 00:06:19,1,Bx7,RIVERDALE AV/W 263 ST,40.912363,-73.902699,WASHINGTON HEIGHTS 166 ST via BROADWAY,40.839813,-73.939745,NYCT_4223,40.866243,-73.925258,BROADWAY/ACADEMY ST,at stop,14,2017-12-01 00:06:21,23:34:35,NaN,NaN
3,2017-12-01 00:05:25,0,S51,LINCOLN AV/RICHMOND RD,40.581245,-74.11199,ST GEORGE FERRY via BAY ST,40.643585,-74.07261,NYCT_7080,40.581364,-74.112033,MIDLAND AV/RICHMOND RD,< 1 stop away,209,2017-12-01 00:06:22,24:15:35,NaN,NaN
4,2017-12-01 00:05:56,0,M8,WEST ST/CHRISTOPHER ST,40.732847,-74.010081,"AVENUE ""D"" CROSSTOWN",40.724689,-73.974548,NYCT_3809,40.734194,-73.999677,GREENWICH AV/CHRISTOPHER ST,at stop,5,2017-12-01 00:06:22,24:05:24,NaN,NaN


In [9]:
transit.drop(columns=18, inplace=True)

In [10]:
## Getting headers
headers = transit.iloc[0]
headers = headers.fillna(value='unknown')
transit = transit.rename(columns=headers)

In [11]:
transit.head()

,RecordedAtTime,DirectionRef,PublishedLineName,OriginName,OriginLat,OriginLong,DestinationName,DestinationLat,DestinationLong,VehicleRef,VehicleLocation.Latitude,VehicleLocation.Longitude,NextStopPointName,ArrivalProximityText,DistanceFromStop,ExpectedArrivalTime,ScheduledArrivalTime,unknown
0,RecordedAtTime,DirectionRef,PublishedLineName,OriginName,OriginLat,OriginLong,DestinationName,DestinationLat,DestinationLong,VehicleRef,VehicleLocation.Latitude,VehicleLocation.Longitude,NextStopPointName,ArrivalProximityText,DistanceFromStop,ExpectedArrivalTime,ScheduledArrivalTime,NaN
1,2017-12-01 00:05:49,0,B67,MC DONALD AV/CORTELYOU RD,40.63816,-73.978939,DNTWN BKLYN SANDS ST via FLATBUSH AV,40.700253,-73.98703,NYCT_406,40.671578,-73.977672,7 AV/GARFIELD PL,approaching,130,2017-12-01 00:06:21,24:01:30,NaN
2,2017-12-01 00:06:19,1,Bx7,RIVERDALE AV/W 263 ST,40.912363,-73.902699,WASHINGTON HEIGHTS 166 ST via BROADWAY,40.839813,-73.939745,NYCT_4223,40.866243,-73.925258,BROADWAY/ACADEMY ST,at stop,14,2017-12-01 00:06:21,23:34:35,NaN
3,2017-12-01 00:05:25,0,S51,LINCOLN AV/RICHMOND RD,40.581245,-74.11199,ST GEORGE FERRY via BAY ST,40.643585,-74.07261,NYCT_7080,40.581364,-74.112033,MIDLAND AV/RICHMOND RD,< 1 stop away,209,2017-12-01 00:06:22,24:15:35,NaN
4,2017-12-01 00:05:56,0,M8,WEST ST/CHRISTOPHER ST,40.732847,-74.010081,"AVENUE ""D"" CROSSTOWN",40.724689,-73.974548,NYCT_3809,40.734194,-73.999677,GREENWICH AV/CHRISTOPHER ST,at stop,5,2017-12-01 00:06:22,24:05:24,NaN


In [12]:
transit=transit[1:]

In [13]:
transit.head()

,RecordedAtTime,DirectionRef,PublishedLineName,OriginName,OriginLat,OriginLong,DestinationName,DestinationLat,DestinationLong,VehicleRef,VehicleLocation.Latitude,VehicleLocation.Longitude,NextStopPointName,ArrivalProximityText,DistanceFromStop,ExpectedArrivalTime,ScheduledArrivalTime,unknown
1,2017-12-01 00:05:49,0,B67,MC DONALD AV/CORTELYOU RD,40.63816,-73.978939,DNTWN BKLYN SANDS ST via FLATBUSH AV,40.700253,-73.98703,NYCT_406,40.671578,-73.977672,7 AV/GARFIELD PL,approaching,130,2017-12-01 00:06:21,24:01:30,NaN
2,2017-12-01 00:06:19,1,Bx7,RIVERDALE AV/W 263 ST,40.912363,-73.902699,WASHINGTON HEIGHTS 166 ST via BROADWAY,40.839813,-73.939745,NYCT_4223,40.866243,-73.925258,BROADWAY/ACADEMY ST,at stop,14,2017-12-01 00:06:21,23:34:35,NaN
3,2017-12-01 00:05:25,0,S51,LINCOLN AV/RICHMOND RD,40.581245,-74.11199,ST GEORGE FERRY via BAY ST,40.643585,-74.07261,NYCT_7080,40.581364,-74.112033,MIDLAND AV/RICHMOND RD,< 1 stop away,209,2017-12-01 00:06:22,24:15:35,NaN
4,2017-12-01 00:05:56,0,M8,WEST ST/CHRISTOPHER ST,40.732847,-74.010081,"AVENUE ""D"" CROSSTOWN",40.724689,-73.974548,NYCT_3809,40.734194,-73.999677,GREENWICH AV/CHRISTOPHER ST,at stop,5,2017-12-01 00:06:22,24:05:24,NaN
5,2017-12-01 00:06:02,0,M101,ASTOR PL/3 AV,40.729567,-73.990052,FT GEORGE 193 ST via 3 AV,40.855666,-73.925259,NYCT_5902,40.773784,-73.957659,3 AV/E 79 ST,approaching,108,2017-12-01 00:06:24,24:03:01,NaN


In [14]:
problems = transit.loc[~transit['unknown'].isna()]

In [15]:
problems.ArrivalProximityText.unique()

array(['for GEO)'], dtype=object)

In [16]:
problems['NextStopPointName']=problems['NextStopPointName']+' '+problems['ArrivalProximityText']
problems['ArrivalProximityText']=problems['DistanceFromStop']
problems['DistanceFromStop']=problems['ExpectedArrivalTime']
problems['ExpectedArrivalTime']=problems['ScheduledArrivalTime']
problems['ScheduledArrivalTime']=problems['unknown']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [17]:
problems.count()

RecordedAtTime               362
DirectionRef                 362
PublishedLineName            362
OriginName                   362
OriginLat                    362
OriginLong                   362
DestinationName              362
DestinationLat               362
DestinationLong              362
VehicleRef                   362
VehicleLocation.Latitude     362
VehicleLocation.Longitude    362
NextStopPointName            362
ArrivalProximityText         362
DistanceFromStop             362
ExpectedArrivalTime          329
ScheduledArrivalTime         362
unknown                      362
dtype: int64

In [18]:
## Replacing problematic rows
transit.loc[problems.index, :] = problems[:]

In [19]:
problems.loc[problems['NextStopPointName']=='CPW/110st (non-public for GEO)']

,RecordedAtTime,DirectionRef,PublishedLineName,OriginName,OriginLat,OriginLong,DestinationName,DestinationLat,DestinationLong,VehicleRef,VehicleLocation.Latitude,VehicleLocation.Longitude,NextStopPointName,ArrivalProximityText,DistanceFromStop,ExpectedArrivalTime,ScheduledArrivalTime,unknown
214530,2017-12-01 19:57:52,1,M10,FRED DOUGLASS BL/W 158 ST,40.830954,-73.93599,COLUMBUS CIRCLE via CENTRAL PARK W.,40.766529,-73.981964,NYCT_6651,40.800822,-73.958138,CPW/110st (non-public for GEO),at stop,14,2017-12-01 19:58:18,19:56:21,19:56:21
228217,2017-12-01 21:17:32,1,M10,FRED DOUGLASS BL/W 158 ST,40.830954,-73.93599,COLUMBUS CIRCLE via CENTRAL PARK W.,40.766529,-73.981964,NYCT_6374,40.800788,-73.958299,CPW/110st (non-public for GEO),at stop,0,NaN,21:14:21,21:14:21
423917,2017-12-03 06:40:30,1,M10,FRED DOUGLASS BL/W 158 ST,40.830954,-73.93599,COLUMBUS CIRCLE via CENTRAL PARK W.,40.766529,-73.981964,NYCT_6663,40.800862,-73.957966,CPW/110st (non-public for GEO),approaching,31,2017-12-03 06:40:47,06:39:38,06:39:38
486256,2017-12-03 15:00:48,1,M10,FRED DOUGLASS BL/W 158 ST,40.830954,-73.93599,COLUMBUS CIRCLE via CENTRAL PARK W.,40.766529,-73.981964,NYCT_6402,40.800798,-73.958251,CPW/110st (non-public for GEO),at stop,4,2017-12-03 15:00:51,14:58:32,14:58:32
523240,2017-12-03 19:01:30,1,M10,FRED DOUGLASS BL/W 158 ST,40.830954,-73.93599,COLUMBUS CIRCLE via CENTRAL PARK W.,40.766529,-73.981964,NYCT_6402,40.800799,-73.958245,CPW/110st (non-public for GEO),at stop,4,2017-12-03 19:01:44,19:03:21,19:03:21
608760,2017-12-04 08:51:48,1,M10,FRED DOUGLASS BL/W 158 ST,40.830954,-73.93599,COLUMBUS CIRCLE via CENTRAL PARK W.,40.766529,-73.981964,NYCT_4405,40.800824,-73.958128,CPW/110st (non-public for GEO),at stop,14,NaN,08:49:54,08:49:54
1108815,2017-12-06 09:27:27,1,M10,FRED DOUGLASS BL/W 158 ST,40.830954,-73.93599,COLUMBUS CIRCLE via CENTRAL PARK W.,40.766529,-73.981964,NYCT_6639,40.800882,-73.957951,CPW/110st (non-public for GEO),approaching,33,2017-12-06 09:27:40,09:28:54,09:28:54
1249595,2017-12-06 19:28:02,1,M10,FRED DOUGLASS BL/W 158 ST,40.830954,-73.93599,COLUMBUS CIRCLE via CENTRAL PARK W.,40.766529,-73.981964,NYCT_3878,40.800788,-73.958299,CPW/110st (non-public for GEO),at stop,0,2017-12-06 19:28:20,19:26:21,19:26:21
1271893,2017-12-06 21:38:19,1,M10,FRED DOUGLASS BL/W 158 ST,40.830954,-73.93599,COLUMBUS CIRCLE via CENTRAL PARK W.,40.766529,-73.981964,NYCT_3878,40.800788,-73.958299,CPW/110st (non-public for GEO),at stop,0,2017-12-06 21:38:28,21:34:21,21:34:21
1324814,2017-12-07 07:48:54,1,M10,FRED DOUGLASS BL/W 158 ST,40.830954,-73.93599,COLUMBUS CIRCLE via CENTRAL PARK W.,40.766529,-73.981964,NYCT_3867,40.800853,-73.957973,CPW/110st (non-public for GEO),at stop,29,2017-12-07 07:49:11,07:45:54,07:45:54


In [20]:
transit.drop(columns='unknown', inplace=True)

In [21]:
problems.loc[problems['NextStopPointName']=='CPW/110st (non-public for GEO)']

,RecordedAtTime,DirectionRef,PublishedLineName,OriginName,OriginLat,OriginLong,DestinationName,DestinationLat,DestinationLong,VehicleRef,VehicleLocation.Latitude,VehicleLocation.Longitude,NextStopPointName,ArrivalProximityText,DistanceFromStop,ExpectedArrivalTime,ScheduledArrivalTime,unknown
214530,2017-12-01 19:57:52,1,M10,FRED DOUGLASS BL/W 158 ST,40.830954,-73.93599,COLUMBUS CIRCLE via CENTRAL PARK W.,40.766529,-73.981964,NYCT_6651,40.800822,-73.958138,CPW/110st (non-public for GEO),at stop,14,2017-12-01 19:58:18,19:56:21,19:56:21
228217,2017-12-01 21:17:32,1,M10,FRED DOUGLASS BL/W 158 ST,40.830954,-73.93599,COLUMBUS CIRCLE via CENTRAL PARK W.,40.766529,-73.981964,NYCT_6374,40.800788,-73.958299,CPW/110st (non-public for GEO),at stop,0,NaN,21:14:21,21:14:21
423917,2017-12-03 06:40:30,1,M10,FRED DOUGLASS BL/W 158 ST,40.830954,-73.93599,COLUMBUS CIRCLE via CENTRAL PARK W.,40.766529,-73.981964,NYCT_6663,40.800862,-73.957966,CPW/110st (non-public for GEO),approaching,31,2017-12-03 06:40:47,06:39:38,06:39:38
486256,2017-12-03 15:00:48,1,M10,FRED DOUGLASS BL/W 158 ST,40.830954,-73.93599,COLUMBUS CIRCLE via CENTRAL PARK W.,40.766529,-73.981964,NYCT_6402,40.800798,-73.958251,CPW/110st (non-public for GEO),at stop,4,2017-12-03 15:00:51,14:58:32,14:58:32
523240,2017-12-03 19:01:30,1,M10,FRED DOUGLASS BL/W 158 ST,40.830954,-73.93599,COLUMBUS CIRCLE via CENTRAL PARK W.,40.766529,-73.981964,NYCT_6402,40.800799,-73.958245,CPW/110st (non-public for GEO),at stop,4,2017-12-03 19:01:44,19:03:21,19:03:21
608760,2017-12-04 08:51:48,1,M10,FRED DOUGLASS BL/W 158 ST,40.830954,-73.93599,COLUMBUS CIRCLE via CENTRAL PARK W.,40.766529,-73.981964,NYCT_4405,40.800824,-73.958128,CPW/110st (non-public for GEO),at stop,14,NaN,08:49:54,08:49:54
1108815,2017-12-06 09:27:27,1,M10,FRED DOUGLASS BL/W 158 ST,40.830954,-73.93599,COLUMBUS CIRCLE via CENTRAL PARK W.,40.766529,-73.981964,NYCT_6639,40.800882,-73.957951,CPW/110st (non-public for GEO),approaching,33,2017-12-06 09:27:40,09:28:54,09:28:54
1249595,2017-12-06 19:28:02,1,M10,FRED DOUGLASS BL/W 158 ST,40.830954,-73.93599,COLUMBUS CIRCLE via CENTRAL PARK W.,40.766529,-73.981964,NYCT_3878,40.800788,-73.958299,CPW/110st (non-public for GEO),at stop,0,2017-12-06 19:28:20,19:26:21,19:26:21
1271893,2017-12-06 21:38:19,1,M10,FRED DOUGLASS BL/W 158 ST,40.830954,-73.93599,COLUMBUS CIRCLE via CENTRAL PARK W.,40.766529,-73.981964,NYCT_3878,40.800788,-73.958299,CPW/110st (non-public for GEO),at stop,0,2017-12-06 21:38:28,21:34:21,21:34:21
1324814,2017-12-07 07:48:54,1,M10,FRED DOUGLASS BL/W 158 ST,40.830954,-73.93599,COLUMBUS CIRCLE via CENTRAL PARK W.,40.766529,-73.981964,NYCT_3867,40.800853,-73.957973,CPW/110st (non-public for GEO),at stop,29,2017-12-07 07:49:11,07:45:54,07:45:54


In [22]:
def missingness_summary(df, print_log=False, sort='ascending'):
  s = df.isnull().sum()*100/df.isnull().count()
    
  if sort.lower() == 'ascending':
    s = s.sort_values(ascending=True)
  elif sort.lower() == 'descending':
    s = s.sort_values(ascending=False)  
  if print_log: 
    print(s)
  
  return pd.Series(s)

In [23]:
missingness_summary(transit)

RecordedAtTime                0.000000
PublishedLineName             0.000000
DestinationName               0.000000
VehicleLocation.Longitude     0.000000
VehicleLocation.Latitude      0.000000
VehicleRef                    0.000000
DistanceFromStop              0.109769
ArrivalProximityText          0.109769
DirectionRef                  4.661862
NextStopPointName             4.767762
DestinationLong               4.892836
DestinationLat                4.892836
OriginLong                    6.077205
OriginLat                     6.077205
OriginName                    6.077205
ScheduledArrivalTime          7.681724
ExpectedArrivalTime          16.453353
dtype: float64

In [24]:
transit.ArrivalProximityText.unique()

array(['approaching', 'at stop', '< 1 stop away', '1.7 miles away',
       '0.6 miles away', '1.3 miles away', '1.2 miles away',
       '5.0 miles away', '2.3 miles away', '0.5 miles away',
       '1.0 miles away', '1.9 miles away', '2.9 miles away',
       '0.8 miles away', nan, '5.6 miles away', '0.9 miles away',
       '2.7 miles away', '1.5 miles away', '1.8 miles away',
       '4.0 miles away', '4.1 miles away', '0.7 miles away',
       '1.1 miles away', '5.3 miles away', '7.0 miles away',
       '3.3 miles away', '6.1 miles away', '4.6 miles away',
       '5.7 miles away', '5.2 miles away', '5.1 miles away',
       '2.5 miles away', '5.8 miles away', '2.6 miles away',
       '3.1 miles away', '2.2 miles away', '4.3 miles away',
       '5.5 miles away', '3.2 miles away', '5.9 miles away',
       '1.4 miles away', '2.0 miles away', '6.5 miles away',
       '6.0 miles away', '2.8 miles away', '6.8 miles away',
       '6.6 miles away', '1.6 miles away', '16.5 miles away',
       '3.8

In [25]:
fixingTheGEOAgain=transit.loc[transit['ArrivalProximityText']=='for GEO)']

In [26]:
fixingTheGEOAgain.count()

RecordedAtTime               7
DirectionRef                 7
PublishedLineName            7
OriginName                   0
OriginLat                    0
OriginLong                   0
DestinationName              7
DestinationLat               7
DestinationLong              7
VehicleRef                   7
VehicleLocation.Latitude     7
VehicleLocation.Longitude    7
NextStopPointName            7
ArrivalProximityText         7
DistanceFromStop             7
ExpectedArrivalTime          7
ScheduledArrivalTime         7
dtype: int64

In [27]:
fixingTheGEOAgain['NextStopPointName']=fixingTheGEOAgain['NextStopPointName']+' '+fixingTheGEOAgain['ArrivalProximityText']
fixingTheGEOAgain['ArrivalProximityText']=fixingTheGEOAgain['DistanceFromStop']
fixingTheGEOAgain['DistanceFromStop']=fixingTheGEOAgain['ExpectedArrivalTime']
fixingTheGEOAgain['ExpectedArrivalTime']=fixingTheGEOAgain['ScheduledArrivalTime']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [28]:
## It should have overflown to the 18th column but we know it didnt so we safely mark it as NaN
fixingTheGEOAgain['ScheduledArrivalTime']=np.nan

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [29]:
fixingTheGEOAgain

,RecordedAtTime,DirectionRef,PublishedLineName,OriginName,OriginLat,OriginLong,DestinationName,DestinationLat,DestinationLong,VehicleRef,VehicleLocation.Latitude,VehicleLocation.Longitude,NextStopPointName,ArrivalProximityText,DistanceFromStop,ExpectedArrivalTime,ScheduledArrivalTime
1675576,2017-12-08 15:50:22,0,M5,NaN,NaN,NaN,LIMITED WASHINGTON HEIGHTS GW BRIDGE,40.848264,-73.937455,NYCT_6631,40.778578,-73.981642,W 72 ST/Amsterdam av ( non-public for GEO),at stop,19,2017-12-08 15:50:29,NaN
2148493,2017-12-11 10:05:09,0,M5,NaN,NaN,NaN,LIMITED WASHINGTON HEIGHTS GW BRIDGE,40.848264,-73.937455,NYCT_6654,40.777999,-73.98194,W 72 ST/Amsterdam av ( non-public for GEO),approaching,90,2017-12-11 10:05:32,NaN
3227060,2017-12-15 15:52:26,0,M5,NaN,NaN,NaN,LIMITED WASHINGTON HEIGHTS GW BRIDGE,40.848264,-73.937455,NYCT_6721,40.776809,-73.981989,W 72 ST/Amsterdam av ( non-public for GEO),< 1 stop away,222,2017-12-15 15:53:49,NaN
3227353,2017-12-15 15:52:38,0,M5,NaN,NaN,NaN,LIMITED WASHINGTON HEIGHTS GW BRIDGE,40.848264,-73.937455,NYCT_3952,40.778443,-73.981739,W 72 ST/Amsterdam av ( non-public for GEO),approaching,37,2017-12-15 15:52:51,NaN
4511202,2017-12-21 15:49:16,0,M5,NaN,NaN,NaN,LIMITED WASHINGTON HEIGHTS GW BRIDGE,40.848264,-73.937455,NYCT_6705,40.777655,-73.981957,W 72 ST/Amsterdam av ( non-public for GEO),approaching,128,2017-12-21 15:50:13,NaN
4754595,2017-12-22 15:30:47,0,M5,NaN,NaN,NaN,LIMITED WASHINGTON HEIGHTS GW BRIDGE,40.848264,-73.937455,NYCT_6635,40.776629,-73.981996,W 72 ST/Amsterdam av ( non-public for GEO),< 1 stop away,242,2017-12-22 15:32:42,NaN
5199544,2017-12-25 12:54:50,1,M10,NaN,NaN,NaN,COLUMBUS CIRCLE via CENTRAL PARK W.,40.766529,-73.981964,NYCT_3851,40.800838,-73.957984,CPW/110st (non-public for GEO),at stop,28,2017-12-25 12:55:06,NaN


In [30]:
transit.loc[fixingTheGEOAgain.index, :] = fixingTheGEOAgain[:]

In [31]:
transit.to_csv('DecemberCleaned.csv', index=False)

In [32]:
transit['PublishedLineName'].unique()

array(['B67', 'Bx7', 'S51', 'M8', 'M101', 'M5', 'B41', 'S53', 'B6', 'Q2',
       'Bx1', 'Bx16', 'Bx41', 'Bx12', 'B35', 'B44', 'B12', 'Bx4',
       'M Shuttle Bus', 'Bx2', 'M57', 'M102', 'S74', 'B63', 'S78', 'B60',
       'M35', 'B46', 'M20', 'B17', 'M50', 'Bx11', 'M116', 'S46',
       'S79-SBS', 'B82', 'Q48', 'Bx15', 'Q4', 'Bx19', 'Q44-SBS', 'S52',
       'S44', 'X10', 'M104', 'M14A', 'Bx28', 'B52', 'B68', 'B2', 'M55',
       'Q58', 'B15', 'Q43', 'X1', 'M11', 'B8', 'Q85', 'M42', 'Bx33',
       'M10', 'Q24', 'B13', 'M60-SBS', 'X17', 'B3', 'B25', 'Q20A', 'Q31',
       'M96', 'M2', 'B16', 'Bx42', 'Bx34', 'M103', 'Q46', 'B57', 'B45',
       'M7', 'M15', 'Bx17', 'S57', 'M66', 'B49', 'B9', 'M1', 'B54', 'B36',
       'M86-SBS', 'M14D', 'B43', 'Q13', 'Q32', 'Q12', 'B38', 'Q3', 'M100',
       'X27', 'B1', 'B4', 'B47', 'Q83', 'B83', 'Bx36', 'B26', 'B61',
       'B62', 'Bx31', 'S66', 'Q27', 'Bx32', 'S40', 'Bx13', 'M31', 'B14',
       'M34A-SBS', 'Bx24', 'Bx9', 'Q59', 'M4', 'B7', 'Bx35', 'Bx40',
 

We have cleaned the dataset, now we have to prepare two datasets,
1. A Bus reaches the bus stop, how much discrepancy is there between the Scheduled time and the actual time
2. A Bus is gonna on the way to a bus stop, we can still predict if it might reach on time, using the distance from bus stop and utilizing the Expected Arrival time. 

Let's go ahead and split the data into two.

## Working with the atTheStop data

In [33]:
coi = ['RecordedAtTime','PublishedLineName', 'DestinationName','VehicleLocation.Latitude', 'VehicleLocation.Longitude','DistanceFromStop','ScheduledArrivalTime']
atthestop = transit[coi].loc[transit.DistanceFromStop == '0'].copy()
atthestop.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 386330 entries, 34 to 6461746
Data columns (total 7 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   RecordedAtTime             386330 non-null  object
 1   PublishedLineName          386330 non-null  object
 2   DestinationName            386330 non-null  object
 3   VehicleLocation.Latitude   386330 non-null  object
 4   VehicleLocation.Longitude  386330 non-null  object
 5   DistanceFromStop           386330 non-null  object
 6   ScheduledArrivalTime       332549 non-null  object
dtypes: object(7)
memory usage: 23.6+ MB


In [34]:
atthestop.dropna().info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 332549 entries, 34 to 6461746
Data columns (total 7 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   RecordedAtTime             332549 non-null  object
 1   PublishedLineName          332549 non-null  object
 2   DestinationName            332549 non-null  object
 3   VehicleLocation.Latitude   332549 non-null  object
 4   VehicleLocation.Longitude  332549 non-null  object
 5   DistanceFromStop           332549 non-null  object
 6   ScheduledArrivalTime       332549 non-null  object
dtypes: object(7)
memory usage: 20.3+ MB


In [35]:
## Rows that sadly give no useful information for our case.
transit[(transit.DistanceFromStop == '0') & (transit.ScheduledArrivalTime.isna())]

,RecordedAtTime,DirectionRef,PublishedLineName,OriginName,OriginLat,OriginLong,DestinationName,DestinationLat,DestinationLong,VehicleRef,VehicleLocation.Latitude,VehicleLocation.Longitude,NextStopPointName,ArrivalProximityText,DistanceFromStop,ExpectedArrivalTime,ScheduledArrivalTime
594,2017-12-01 00:06:12,1,Q83,227 ST/113 DR,40.702262,-73.730337,JAMAICA HILLSIDE - 153 via LIBERTY,40.706795,-73.804099,NYCT_6871,40.706745,-73.804277,153 ST/HILLSIDE AV,at stop,0,NaN,NaN
766,2017-12-01 00:16:36,0,Q24,PATCHEN AV/LAFAYETTE AV,40.692479,-73.927904,JAMAICA 168 ST via ATLANTIC,40.704442,-73.792612,NYCT_6836,40.704497,-73.792646,ARCHER AV/MERRICK BL,at stop,0,NaN,NaN
803,2017-12-01 00:16:27,0,S79-SBS,MARSH AV/RING RD,40.583223,-74.161951,SELECT BUS BAY RIDGE 86 ST STATION,40.622521,-74.028343,NYCT_4249,40.622545,-74.028453,4 AV/86 ST,at stop,0,NaN,NaN
856,2017-12-01 00:16:39,0,S66,PARK AV/RICHMOND TE,40.639846,-74.130695,ST GEORGE FERRY via GRYMES HILL,40.643356,-74.072914,NYCT_8191,40.643284,-74.072882,ST GEORGE FERRY/S66,at stop,0,NaN,NaN
900,2017-12-01 00:16:36,1,B70,39 ST/1 AV,40.656455,-74.012243,DYKER HEIGHTS VA HOSPITAL via 8 AV,40.609134,-74.023885,NYCT_742,40.609184,-74.023834,CROPSEY AV/BKLYN CAMPUS VA,at stop,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6461165,2017-12-31 23:45:35,0,Bx10,E 206 ST/BAINBRIDGE AV,40.875008,-73.880139,RIVERDALE 263 ST,40.912377,-73.902533,NYCT_8367,40.912389,-73.90261,RIVERDALE AV/W 263 ST,at stop,0,NaN,NaN
6461225,2017-12-31 23:55:54,0,Q44-SBS,MERRICK BL/ARCHER AV,40.704907,-73.793306,SELECT BUS BRONX ZOO via MAIN ST,40.842458,-73.877118,NYCT_6019,40.84239,-73.877142,BOSTON RD/E 180 ST,at stop,0,NaN,NaN
6461428,2017-12-31 23:55:47,1,M60-SBS,LGA/TERMINAL D,40.768386,-73.862535,SELECT BUS WEST SIDE BWAY - 106 ST,40.801424,-73.968241,NYCT_6102,40.801483,-73.9682,W 106 ST/BROADWAY,at stop,0,2017-12-31 23:55:56,NaN
6461524,2017-12-31 23:55:25,0,M23-SBS,12 AV/W 23 ST,40.748717,-74.00811,SELECT BUS EAST SIDE AVENUE C CROSSTOWN,40.733004,-73.974597,NYCT_5930,40.733007,-73.974577,AV C/E 20 ST,at stop,0,NaN,NaN


In [36]:
atthestop.dropna(inplace=True)

In [37]:
## Not needed anymore as well
atthestop.drop(columns='DistanceFromStop', inplace=True)

In [38]:
atthestop['Lat'] = atthestop['VehicleLocation.Latitude'].apply(pd.to_numeric, errors='coerce')
atthestop['Long'] = atthestop['VehicleLocation.Longitude'].apply(pd.to_numeric, errors='coerce')
atthestop.drop(columns=['VehicleLocation.Latitude', 'VehicleLocation.Longitude'], inplace=True)
atthestop

,RecordedAtTime,PublishedLineName,DestinationName,ScheduledArrivalTime,Lat,Long
34,2017-12-01 00:06:01,M35,WARDS ISLAND,24:15:00,40.804532,-73.937305
35,2017-12-01 00:06:04,B46,KINGS PLAZA via UTICA,23:57:39,40.695575,-73.932746
40,2017-12-01 00:06:01,M116,WEST SIDE BWAY - 106 ST XTOWN,24:05:00,40.797200,-73.931587
42,2017-12-01 00:05:50,S79-SBS,SELECT BUS SI MALL via HYLAN,24:02:46,40.566829,-74.113534
79,2017-12-01 00:06:04,Q4,CAMBRIA HTS 235 ST via LINDEN,23:58:00,40.690825,-73.727576
...,...,...,...,...,...,...
6461711,2017-12-31 23:55:46,M14D,LOWER E. SIDE DELANCY-FDR CROSSTOWN,23:57:00,40.737413,-74.004981
6461717,2017-12-31 23:55:39,S62,ST GEORGE FERRY via VICTORY,23:54:55,40.608761,-74.153798
6461742,2017-12-31 23:55:24,Bx9,RIVERDALE 262 ST,23:52:25,40.853235,-73.881700
6461743,2017-12-31 23:55:47,B70,SUNSET PARK 39ST-1AV via 8 AV,23:42:25,40.655498,-74.010867


In [39]:
atthestop['RecordedAtTime'] = pd.to_datetime(atthestop['RecordedAtTime'], errors = 'coerce')
tindex = pd.DatetimeIndex(atthestop.RecordedAtTime) 
#Recorded Year, Day, and Month as a separate column for convenience
atthestop['RecordDate'] = tindex.strftime('%Y-%m-%d ')

In [40]:
sTimes = atthestop[['ScheduledArrivalTime', 'RecordDate']].loc[atthestop.ScheduledArrivalTime >= "24"]

In [41]:
ymd = sTimes.RecordDate
hours = sTimes.ScheduledArrivalTime.str[0:2]
rest = sTimes.ScheduledArrivalTime.str[2:]
# subtract 24
hours = hours.astype(int) - 24
# reassign the string
converted = hours.astype(str) + rest
converted = ymd + converted
# make it datetime, adding one to the day because it's after midnight. 
converted = pd.to_datetime(converted, format = '%Y-%m-%d %H:%M:%S')
converted = converted

In [42]:
atthestop['scheduled'] = pd.to_datetime(atthestop['RecordDate'] + atthestop['ScheduledArrivalTime'], format = '%Y-%m-%d %H:%M:%S', errors='coerce')
atthestop['scheduled'].fillna(value=converted, inplace=True)

#correct for situations after midnight. 
condition = (atthestop.scheduled - atthestop.RecordedAtTime).astype('timedelta64[m]') >= 1000
atthestop.loc[condition, ['scheduled']] = atthestop.loc[condition, ['scheduled']] - pd.Timedelta(days=1)

condition = (atthestop.scheduled - atthestop.RecordedAtTime).astype('timedelta64[m]') <= -1000
atthestop.loc[condition, ['scheduled']] = atthestop.loc[condition, ['scheduled']] + pd.Timedelta(days=1)

#late indicator
condition = (atthestop.scheduled - atthestop.RecordedAtTime).astype('timedelta64[m]') < 0
atthestop['late'] = condition


#adherence
atthestop['adherence'] = (atthestop.scheduled - atthestop.RecordedAtTime).astype('timedelta64[m]') 

#Look at it
atthestop[['scheduled', 'RecordedAtTime','late', 'adherence']]

,scheduled,RecordedAtTime,late,adherence
34,2017-12-01 00:15:00,2017-12-01 00:06:01,False,8.0
35,2017-11-30 23:57:39,2017-12-01 00:06:04,True,-9.0
40,2017-12-01 00:05:00,2017-12-01 00:06:01,True,-2.0
42,2017-12-01 00:02:46,2017-12-01 00:05:50,True,-4.0
79,2017-11-30 23:58:00,2017-12-01 00:06:04,True,-9.0
...,...,...,...,...
6461711,2017-12-31 23:57:00,2017-12-31 23:55:46,False,1.0
6461717,2017-12-31 23:54:55,2017-12-31 23:55:39,True,-1.0
6461742,2017-12-31 23:52:25,2017-12-31 23:55:24,True,-3.0
6461743,2017-12-31 23:42:25,2017-12-31 23:55:47,True,-14.0


In [43]:
atthestop

,RecordedAtTime,PublishedLineName,DestinationName,ScheduledArrivalTime,Lat,Long,RecordDate,scheduled,late,adherence
34,2017-12-01 00:06:01,M35,WARDS ISLAND,24:15:00,40.804532,-73.937305,2017-12-01,2017-12-01 00:15:00,False,8.0
35,2017-12-01 00:06:04,B46,KINGS PLAZA via UTICA,23:57:39,40.695575,-73.932746,2017-12-01,2017-11-30 23:57:39,True,-9.0
40,2017-12-01 00:06:01,M116,WEST SIDE BWAY - 106 ST XTOWN,24:05:00,40.797200,-73.931587,2017-12-01,2017-12-01 00:05:00,True,-2.0
42,2017-12-01 00:05:50,S79-SBS,SELECT BUS SI MALL via HYLAN,24:02:46,40.566829,-74.113534,2017-12-01,2017-12-01 00:02:46,True,-4.0
79,2017-12-01 00:06:04,Q4,CAMBRIA HTS 235 ST via LINDEN,23:58:00,40.690825,-73.727576,2017-12-01,2017-11-30 23:58:00,True,-9.0
...,...,...,...,...,...,...,...,...,...,...
6461711,2017-12-31 23:55:46,M14D,LOWER E. SIDE DELANCY-FDR CROSSTOWN,23:57:00,40.737413,-74.004981,2017-12-31,2017-12-31 23:57:00,False,1.0
6461717,2017-12-31 23:55:39,S62,ST GEORGE FERRY via VICTORY,23:54:55,40.608761,-74.153798,2017-12-31,2017-12-31 23:54:55,True,-1.0
6461742,2017-12-31 23:55:24,Bx9,RIVERDALE 262 ST,23:52:25,40.853235,-73.881700,2017-12-31,2017-12-31 23:52:25,True,-3.0
6461743,2017-12-31 23:55:47,B70,SUNSET PARK 39ST-1AV via 8 AV,23:42:25,40.655498,-74.010867,2017-12-31,2017-12-31 23:42:25,True,-14.0


In [44]:
transit_atTheStop=transit.loc[atthestop.index]

In [45]:
transit_atTheStop=pd.merge(transit_atTheStop, atthestop[['scheduled', 'RecordedAtTime','late', 'adherence']], left_index=True, right_index=True)

In [46]:
transit_atTheStop.loc[~(transit_atTheStop['RecordedAtTime_x']==transit_atTheStop['RecordedAtTime_y'].astype(str))][['RecordedAtTime_y','RecordedAtTime_x']]

,RecordedAtTime_y,RecordedAtTime_x


In [47]:
transit_atTheStop.head()

,RecordedAtTime_x,DirectionRef,PublishedLineName,OriginName,OriginLat,OriginLong,DestinationName,DestinationLat,DestinationLong,VehicleRef,VehicleLocation.Latitude,VehicleLocation.Longitude,NextStopPointName,ArrivalProximityText,DistanceFromStop,ExpectedArrivalTime,ScheduledArrivalTime,scheduled,RecordedAtTime_y,late,adherence
34,2017-12-01 00:06:01,0,M35,LEXINGTON AV/E 125 ST,40.80458,-73.937417,WARDS ISLAND,40.785111,-73.930715,NYCT_1269,40.804532,-73.937305,LEXINGTON AV/E 125 ST,at stop,0,NaN,24:15:00,2017-12-01 00:15:00,2017-12-01 00:06:01,False,8.0
35,2017-12-01 00:06:04,1,B46,WASHINGTON PLZ/LANE3,40.709392,-73.95952,KINGS PLAZA via UTICA,40.60914,-73.921443,NYCT_5139,40.695575,-73.932746,BROADWAY/WILLOUGHBY AV,at stop,0,NaN,23:57:39,2017-11-30 23:57:39,2017-12-01 00:06:04,True,-9.0
40,2017-12-01 00:06:01,1,M116,PLEASANT AV/E 120 ST,40.797246,-73.931695,WEST SIDE BWAY - 106 ST XTOWN,40.801329,-73.967601,NYCT_3880,40.7972,-73.931587,PLEASANT AV/E 120 ST,at stop,0,NaN,24:05:00,2017-12-01 00:05:00,2017-12-01 00:06:01,True,-2.0
42,2017-12-01 00:05:50,1,S79-SBS,4 AV/86 ST,40.622658,-74.028343,SELECT BUS SI MALL via HYLAN,40.583351,-74.161813,NYCT_4272,40.566829,-74.113534,HYLAN BL/LINDBERGH AV,at stop,0,2017-12-01 00:06:22,24:02:46,2017-12-01 00:02:46,2017-12-01 00:05:50,True,-4.0
79,2017-12-01 00:06:04,0,Q4,TEARDROP/LAYOVER,40.701747,-73.802398,CAMBRIA HTS 235 ST via LINDEN,40.690804,-73.727581,NYCT_4910,40.690825,-73.727576,235 ST / LINDEN BL,at stop,0,2017-12-01 00:06:22,23:58:00,2017-11-30 23:58:00,2017-12-01 00:06:04,True,-9.0


In [48]:
transit_atTheStop.drop(columns='RecordedAtTime_x',inplace=True)

In [49]:
transit_atTheStop = transit_atTheStop.rename({'RecordedAtTime_y': 'RecordedAtTime'}, axis='columns')


In [50]:
transit_atTheStop.to_csv('DecemberCleanedV2.csv', index=False)

## Now the delta of Scheduled and Arrival times

In [51]:
transit.dropna(subset=['RecordedAtTime', 'ScheduledArrivalTime','ExpectedArrivalTime'],inplace=True)

In [52]:
sTimes[sTimes['ScheduledArrivalTime'].str.match('^s.*')== False]

,ScheduledArrivalTime,RecordDate
34,24:15:00,2017-12-01
40,24:05:00,2017-12-01
42,24:02:46,2017-12-01
81,24:06:25,2017-12-01
105,24:03:54,2017-12-01
...,...,...
6461362,24:00:00,2017-12-31
6461379,24:08:00,2017-12-31
6461548,24:00:00,2017-12-31
6461566,24:00:00,2017-12-31


In [53]:
transit['RecordedAtTime'] = pd.to_datetime(transit['RecordedAtTime'], errors = 'coerce')
transit['ExpectedArrivalTime'] = pd.to_datetime(transit['ExpectedArrivalTime'], errors = 'coerce')

tindex = pd.DatetimeIndex(transit.RecordedAtTime) 
#Recorded Year, Day, and Month as a separate column for convenience
transit['RecordDate'] = tindex.strftime('%Y-%m-%d ')

## The troubling dates
sTimes = transit[['ScheduledArrivalTime', 'RecordDate']].loc[transit.ScheduledArrivalTime >= "24"]
ymd = sTimes.RecordDate
hours = sTimes.ScheduledArrivalTime.str[0:2]
rest = sTimes.ScheduledArrivalTime.str[2:]
# subtract 24
hours = hours.astype(int) - 24
# reassign the string
converted = hours.astype(str) + rest
converted = ymd + converted
# make it datetime, adding one to the day because it's after midnight. 
converted = pd.to_datetime(converted, format = '%Y-%m-%d %H:%M:%S')
converted = converted

transit['scheduled'] = pd.to_datetime(transit['RecordDate'] + transit['ScheduledArrivalTime'], format = '%Y-%m-%d %H:%M:%S', errors='coerce')
transit['scheduled'].fillna(value=converted, inplace=True)

#correct for situations after midnight. 
condition = (transit.scheduled - transit.RecordedAtTime).astype('timedelta64[m]') >= 1000
transit.loc[condition, ['scheduled']] = transit.loc[condition, ['scheduled']] - pd.Timedelta(days=1)

condition = (transit.scheduled - transit.RecordedAtTime).astype('timedelta64[m]') <= -1000
transit.loc[condition, ['scheduled']] = transit.loc[condition, ['scheduled']] + pd.Timedelta(days=1)

#late indicator
Expcondition = (transit.scheduled - transit.ExpectedArrivalTime).astype('timedelta64[m]') < 0
transit['ExpectedLateness'] = Expcondition


#adherence
transit['ExpectedAdherence'] = (transit.scheduled - transit.ExpectedArrivalTime).astype('timedelta64[m]') 

In [54]:
transit.to_csv('DecemberCleanedV3.csv', index=False)